### Definición Formal del Coeficiente de Correlación de Gideon o de Mayor Desviación (GDCC).


El GDCC es un coeficiente de correlación que mide la relación entre dos variables aleatorias bivariadas. A diferencia de otros coeficientes de correlación, como Pearson o Kendall, el GDCC se enfoca en ser robusto frente a valores atípicos o extremos en los datos.

El GDCC se basa en la idea de que la correlación entre dos variables puede ser influenciada significativamente por valores atípicos. En lugar de utilizar todos los puntos de datos para calcular la correlación, el GDCC se concentra en el punto de datos que presenta la mayor desviación de una línea de regresión hipotética, haciendo que el coeficiente sea menos sensible a valores atípicos y, por lo tanto, más robusto.

**Definición Matemática:**
Dada una muestra de tamaño $ n $ y una permutación $ p_i $ de rangos de una variable dependiente asociada a cada valor de una variable independiente, el GDCC, denotado como $ r_{gd} $, se define como:

$$ r_{gd} = \frac{\max\limits_{1 \leq i \leq n} \left( \sum_{j=1}^{i} I(n+1+p_j > i) \right) - \max\limits_{1 \leq i \leq n} \left( \sum_{j=1}^{i} I(p_j > i) \right)}{\left\lfloor \frac{n}{2} \right\rfloor} $$

Donde:
- $ I $ es la función indicadora, que toma el valor 1 si la condición dentro de la función es verdadera y 0 si es falsa.
- El denominador $ \left\lfloor \frac{n}{2} \right\rfloor $ es la función del mayor entero menor o igual a $ n/2 $.

**Interpretación:**
- Valores de $ r_{gd} $ cercanos a 1 indican una fuerte correlación positiva.
- Valores cercanos a -1 indican una fuerte correlación negativa.
- Valores cercanos a 0 indican una ausencia de correlación o una correlación muy débil.


### Algoritmo para Calcular el (GDCC)

**Entrada:**
- Una lista $P$ de longitud $N$ que representa la permutación $p_i$ (los rangos de la variable dependiente correspondientes a cada valor de la variable independiente).

**Salida:**
- El valor del GDCC, denotado como $r_{gd}$.

**Pasos:**
1. **Inicializar Contadores:** 
   - Crear dos listas, $I\_sum\_R$ e $I\_sum\_L$, ambas de longitud $N$.
   
2. **Calcular las Sumas Indicadoras ($I\_sum\_R$ e $I\_sum\_L$):**
   - Para cada $i$ desde 1 hasta $N$:
     - Calcular $I\_sum\_R[i]$ como la suma de 1 para cada $j$ desde 1 hasta $i$ donde $p_j > i$.
     - Calcular $I\_sum\_L[i]$ como la suma de 1 para cada $j$ desde 1 hasta $i$ donde $N + 1 - p_j > i$.

3. **Encontrar los Valores Máximos de las Sumas Indicadoras:**
   - Encontrar el valor máximo de $I\_sum\_R$, denotado como $max\_I\_sum\_R$.
   - Encontrar el valor máximo de $I\_sum\_L$, denotado como $max\_I\_sum\_L$.

4. **Calcular el GDCC:**
   - Calcular $r_{gd}$ usando la fórmula:
     $$ r_{gd} = \frac{max\_I\_sum\_L - max\_I\_sum\_R}{\left\lfloor \frac{N}{2} \right\rfloor} $$

5. **Retornar el Valor de GDCC:**
   - Retornar $r_{gd}$.


Para más información, consulte el artículo en [https://www.tandfonline.com/doi/abs/10.1080/01621459.1987.10478480](https://www.tandfonline.com/doi/abs/10.1080/01621459.1987.10478480).

# Implementación:

In [2]:
import pandas as pd
import numpy as np
from itertools import combinations

def calculate_gdcc(p_values):
    """
    Calcula el GDCC para un conjunto de datos univariado.
    
    Args:
    p_values (list): Lista de valores para los cuales calcular el GDCC.

    Returns:
    float: El valor del GDCC para el conjunto de datos.
    """
    N = len(p_values)  # Tamaño del conjunto de datos
    # Calcula la suma de indicadores para cada punto, considerando el lado derecho de la fórmula del GDCC
    I_sum_R = [sum(1 for pj in p_values[:i] if pj > i) for i in range(1, N + 1)]
    # Calcula la suma de indicadores para cada punto, considerando el lado izquierdo de la fórmula del GDCC
    I_sum_L = [sum(1 for pj in p_values[:i] if N + 1 - pj > i) for i in range(1, N + 1)]

    # Encuentra los valores máximos de las sumas de indicadores de ambos lados
    max_I_sum_L = max(I_sum_L)
    max_I_sum_R = max(I_sum_R)
    # Calcula el GDCC usando la fórmula dada
    rgd = (max_I_sum_L - max_I_sum_R) / (N // 2)
    return rgd

def calculate_gdcc_for_dataframe(df):
    """
    Calcula el GDCC para cada par de variables en un DataFrame multivariado.

    Args:
    df (DataFrame): DataFrame de Pandas con los datos multivariados.

    Returns:
    DataFrame: Una matriz de correlación GDCC.
    """
    columns = df.columns  # Obtiene los nombres de las columnas del DataFrame
    # Crea una matriz de correlación GDCC vacía
    gdcc_matrix = pd.DataFrame(np.nan, index=columns, columns=columns)

    # Calcula el GDCC para cada combinación de pares de columnas
    for col1, col2 in combinations(columns, 2):
        # Obtiene los rangos para las columnas seleccionadas
        ranks_col1 = df[col1].rank(method='average')
        ranks_col2 = df[col2].rank(method='average')
        # Calcula el valor GDCC para el par de columnas y lo asigna a la matriz
        gdcc_value = calculate_gdcc(ranks_col2.values)
        gdcc_matrix.at[col1, col2] = gdcc_value
        gdcc_matrix.at[col2, col1] = gdcc_value

    # Calcula y asigna el GDCC para cada variable consigo misma
    for col in columns:
        gdcc_self = calculate_gdcc(df[col])
        gdcc_matrix.at[col, col] = gdcc_self

    return gdcc_matrix

In [3]:
# Conjunto de datos YMCA indicado en el articulo.

Variable1 = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
Variable2 = [14,11,16,2,12,13,7,9,10,3,8,1,15,6,4,5]

df_example = pd.DataFrame({
    'Variable1': Variable1,
    'Variable2': Variable2
})

gdcc_matrix  = calculate_gdcc_for_dataframe(df_example)

gdcc_matrix

,Variable1,Variable2
Variable1,1.000,-0.375
Variable2,-0.375,-0.375
